<center> <h1>📉 Logistic Regression 📉 </h1> </center>

<p> <center> This notebook is in <span style="color: green"> <b> Active </b> </span> state of development! </center> </p>  
<p> <center> Be sure to checkout my other notebooks for <span style="color: blue"> <b> knowledge, insight and laughter </b> </span>! 🧠💡😂</center> </p> 

<center> <img src="https://i.redd.it/84w2yii8tcs31.png" width="400" height="400" /> </center>

<hr style="height:2px;border-width:0;color:gray;background-color:gray">

# Aim

The aim is to provide, from scratch, code implementations for linear regression problems. This will involve both the main functions needed to solve a linear regression and some additional utility functions as well.

**Note**: We will not be diving into in-depth exploratory data analysis, feature engineering etc... in these notebooks and so will not be commenting extensively on things such as skewness, kurtosis, homoscedasticity etc...

<hr style="height:2px;border-width:0;color:gray;background-color:gray">

# Background

Logistic regression, despite its name, is a linear model for classification rather than regression. The reason why the term logistic *regression* is used becomes obvious once we examine the logistic function (often also called sigmoid function):
$$g(z) = \frac{1}{1+e^{-z}}$$

Recall that the logistic regression hypothesis is defined as:

$$ h_\theta(\underline{x}) = g(\underline{x}^T\underline{\theta})$$

where the function $g$ is the sigmoid function.

Types of Logistic Regression:

1. Binary (Pass/Fail)
2. Multi (Cats, Dogs, Sheep)
3. Ordinal (Low, Medium, High)

<hr style="height:2px;border-width:0;color:gray;background-color:gray">

## Import Modules

In [ ]:
# Importing standard packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Dict, List, Tuple
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, r2_score

## Logistic Function (Sigmoid)

In [ ]:
def logistic_func(z: List) -> np.array:
        
    """ Calculate logistic/sigmoid function . """

    return 1/(1+np.exp(-np.array(z)))

## Decision Boundary

Our current prediction/logistic function returns a probability score between 0 and 1. In order to map this to a discrete class (true/false, cat/dog), we select a threshold value or tipping point above which we will classify values into class 1 and below which we classify values into class 0.

$$p≥0.5,𝑐𝑙𝑎𝑠𝑠=1$$
$$p<0.5,𝑐𝑙𝑎𝑠𝑠=0$$

For example, if our threshold was 0.5 and our prediction function returned 0.7, we would classify this observation as positive (class 1). If our prediction was 0.2 we would classify the observation as negative (class 0). For logistic regression with multiple classes we could select the class with the highest predicted probability.

In [ ]:
# Plot logistic function
plt.figure(figsize=(8,6))
x = np.linspace(-6, 6, 1000)
y = logistic_func(x)
# Plot decision boundary
x1, y1 = [-6, 6], [0.5, 0.5]
plt.xlabel(r'$x$', size=20)
plt.ylabel(r'$y$', size=20)
plt.title('Logistic Function (Sigmoid)')
plt.grid(alpha=0.5)
plt.plot(x, y);
plt.plot(x1, y1, marker = 'o');
plt.legend(labels=["Sigmoid Function", "Decision Boundary"]);

# Data Collection

Depending on the type of data, certain metrics will be more important than others. Some examples are:
1. Maximise recall of model i.e. reduce the number of misclassifications of fraud cases (Type 2 errors/False Negatives).
2. Maximise precision of model i.e. reduce the number of misclassifications of non-fraud cases (Type 1 errors/False Positives)

We would like a balanced trade off between these metrics - this is encompassed in the F Score. 

**Note:** Maximising accuracy of the model can also be valuable information as it tells us how many datapoints we are correctly classifying however if we had a large imbalanced dataset, this is not necessarily reflective of our needs. 

In [ ]:
# Import dataset
df = pd.read_csv('../input/heart-disease-prediction-using-logistic-regression/framingham.csv')
# Display dataframe
df

# Data Processing

In [ ]:
# Check for Nulls
df.info()

In [ ]:
# Check for NaNs 
df.isna().sum()

In [ ]:
# Check for duplicates
df.duplicated().sum()

# Data Visualisation

Let us look at the information obtained **before** applying the pre-processing steps:

In [ ]:
# Overall statistics
df.describe()

In [ ]:
# Bar chart of class ratio 
target_pd = pd.DataFrame(index = ["No Disease","Disease"], columns= ["Quantity", "Percentage"])
# No disease
target_pd.loc["No Disease"]["Quantity"] = len(df[df.columns[-1]][df[df.columns[-1]]==0].dropna())
target_pd.loc["No Disease"]["Percentage"] = target_pd.iloc[0,0]/len(df[df.columns[-1]])*100
# Disease
target_pd.loc["Disease"]["Quantity"] = len(df[df.columns[-1]][df[df.columns[-1]]==1].dropna())
target_pd.loc["Disease"]["Percentage"] = target_pd.iloc[1,0]/len(df[df.columns[-1]])*100
# Plot barchart
fig = plt.figure(figsize = (10, 5))
plt.bar(list(target_pd.index), target_pd.iloc[:,0], color ='maroon',width = 0.4)
plt.ylabel("Number of cases")
plt.title("Distribution of disease and non-disease cases");
# Print the dataframe
target_pd

In [ ]:
# Histogram of features (check for skew)
fig=plt.figure(figsize=(20,20))
for i, feature in enumerate(df.columns):
    ax=fig.add_subplot(8,4,i+1)
    df[feature].hist(bins=20,ax=ax,facecolor='black')
    ax.set_title(feature+" Distribution",color='DarkRed')
    ax.set_yscale('log')
fig.tight_layout()  

In [ ]:
# Check for correlation
fig=plt.figure(figsize=(20,20))
sns.heatmap(df.corr(), annot = True, cmap="tab20c");
fig.tight_layout()  

Let us look at the information obtained **after** applying the pre-processing steps:

In [ ]:
# Drop NaNs and duplicates
df = df.dropna().drop_duplicates().reset_index(drop=True)

In [ ]:
# Overall statistics
df.describe()

In [ ]:
# Bar chart of class ratio 
target_pd = pd.DataFrame(index = ["No Disease","Disease"], columns= ["Quantity", "Percentage"])
# No disease
target_pd.loc["No Disease"]["Quantity"] = len(df[df.columns[-1]][df[df.columns[-1]]==0].dropna())
target_pd.loc["No Disease"]["Percentage"] = target_pd.iloc[0,0]/len(df[df.columns[-1]])*100
# Disease
target_pd.loc["Disease"]["Quantity"] = len(df[df.columns[-1]][df[df.columns[-1]]==1].dropna())
target_pd.loc["Disease"]["Percentage"] = target_pd.iloc[1,0]/len(df[df.columns[-1]])*100
# Plot barchart
fig = plt.figure(figsize = (10, 5))
plt.bar(list(target_pd.index), target_pd.iloc[:,0], color ='maroon',width = 0.4)
plt.ylabel("Number of cases")
plt.title("Distribution of disease and non-disease cases");
# Print the dataframe
target_pd

In [ ]:
# Histogram of features (check for skew)
fig=plt.figure(figsize=(20,20))
for i, feature in enumerate(df.columns):
    ax=fig.add_subplot(8,4,i+1)
    df[feature].hist(bins=20,ax=ax,facecolor='black')
    ax.set_title(feature+" Distribution",color='DarkRed')
    ax.set_yscale('log')
fig.tight_layout()  

In [ ]:
# Check for correlation
fig=plt.figure(figsize=(20,20))
sns.heatmap(df.corr(), annot = True, cmap="tab20c");
fig.tight_layout()  

We can see not much has changed from the overall statistics of the data after pre-processing which is excellent! We can no proceed with machine learning models.

**Note:** Instead of just removing the NaNs, we could have imputed them with different techniques. 

## Splitting dataset

For most machine learning models, we would like them to have low bias and low variance - that is, the model should perform well on the training set (low bias) and also the test set, alongside with other new random test sets (low variance). Therefore, to test for bias and variance of our model, we shall split the dataset into training and test set. We will not be tuning any hyperparameters (and thus do not need a validation set).  We will not be tuning any hyperparameters (and thus do not need a validation set). 

For these functions, the $X$ dataset (of features) should have a column 1's as the first column to account for the bias term/intercept co-efficient. Before this occurs, one should check the order of magnitude of the features - if they differ hugely, one must apply feature scaling. Having looked at the data however, it is clear that the order of magnitude of some of the features are very different, so we must perform feature scaling. 

In [ ]:
# Create X (features) and y (target) dataset
X = df[df.columns[:-1]]
y = df[df.columns[-1]]

In [ ]:
def feature_scaling(X: pd.DataFrame) -> pd.DataFrame:
    
    """ Normalises the features in X (dataframe) and returns a normalized version of X where the mean value of each feature is 0 and the standard deviation is 1. """
    
    # Return normalised data
    return (X - np.mean(X, axis=0))/np.std(X, axis=0, ddof=0)

In [ ]:
# Create normalised data
X = feature_scaling(X)
# Split the dataset into training and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1/4, random_state=42, stratify=y)
# Re-index
X_train = X_train.reset_index(drop=True) 
y_train = y_train.reset_index(drop=True) 
X_test = X_test.reset_index(drop=True) 
y_test = y_test.reset_index(drop=True) 

In [ ]:
# Add ones to the dataframes
m,n = X_train.values.shape
o,p = X_test.values.shape
X_train = pd.concat((pd.DataFrame(np.ones((m, 1)), columns= ['Bias']),X_train),axis=1)
X_test = pd.concat((pd.DataFrame(np.ones((o, 1)), columns= ['Bias']),X_test),axis=1)

# Cost Function

Recall that the cost function (known as cross entropy) in logistic regression is:

$$ G(\boldsymbol{\theta}) = \frac{1}{n} \sum_{i=1}^{n} Cost\left(h_\theta\left( x^{(i)} \right), y^{(i)} \right)$$
where
$$ Cost\left(h_\theta\left( x \right), y \right) = \log\left(h_\theta\left( x \right) \right) \qquad \text{if} \ y=1$$
$$ Cost\left(h_\theta\left( x \right), y \right) = -\log\left(1-h_\theta\left( x \right) \right) \qquad \text{if} \ y=0$$

Compressing these events together into one (by treating the predictions as Bernoulli random variable events), we obtain:

$$ G(\boldsymbol{\theta}) = \pm \frac{1}{n} \sum_{i=1}^{n} \left[ y^{(i)} \log\left(h_\theta\left( x^{(i)} \right) \right) + \left( 1 - y^{(i)}\right) \log \left( 1 - h_\theta\left( x^{(i)} \right) \right) \right]$$

**Note:** Depending on visualisation, we'd like our cost function to be positive so in this case we will take $G(\boldsymbol \theta)$ to take on the negative argument.

# Gradient Function

Through partial differentiation, the gradient of the cost is a vector of the same length as $\theta$ where the $j^{th}$
element (for $j = 0, 1, \cdots , n$) is defined as follows:

$$ \frac{\partial G(\boldsymbol\theta)}{\partial \theta_j} = \frac{1}{n} \sum_{i=1}^n \left( h_\theta \left( x^{(i)} \right) - y^{(i)} \right) x_j^{(i)} $$

**Note:** While this gradient looks identical to the linear regression gradient, the formula is actually different because linear and logistic regression have different definitions of $h_\theta(x)$.

## Vectorised notation

It is easier to use vectorised notation to code this:

$$ h_\theta\left( x \right) = g\left(X \theta \right) \qquad \text{logistic regression hypothesis} $$
$$ G(\boldsymbol{\theta}) = \pm \frac{1}{n} \cdot \left(y^T log\left(h\right) - \left(1-y\right)^T log\left(1-h\right) \right) \qquad \text{cost function} $$
$$ \frac{\partial G(\boldsymbol\theta)}{\partial \theta} = \frac{1}{n} \cdot X^T(h-y) \qquad \text{gradient function} $$

In [ ]:
def LR_CG(X: pd.DataFrame, y: pd.Series, theta: np.array) -> Tuple[Dict, float]:
    
    """ Calculate the cost and gradients of the logistic model via gradient descent. """

    n = X.shape[0]
    h = self.logistic_func(X @ theta)
    # Cost function (add minus for appropriacy)
    cost = -1/n * (y.T @ np.log(h) + (1 - y).T @ np.log(1-h)) 
    # Derivatives
    dtheta = 1/n * (X.T @ (h-y))
    # Store gradients in a dictionary
    grads = {"dtheta": dtheta}
    return grads, cost

# Batch Gradient Descent

Recall that the parameters of our model are the $\theta_j$ values. These are
the values you will adjust to minimize cost $G(\underline{\theta})$. One way to do this is to
use the batch gradient descent algorithm. In batch gradient descent, each
iteration performs the update

$$ \theta_j = \theta_j - \alpha \frac{\partial G(\boldsymbol\theta)}{\partial \theta_j} \qquad \text{simultaneously update } \theta_j \text{ for all } j$$

With each step of gradient descent, your parameters $\theta_j$ come closer to the optimal values that will achieve the lowest cost $G(\boldsymbol \theta)$ (see p.g. 10 [Logistic Regression](https://web.stanford.edu/~jurafsky/slp3/5.pdf)).

In [ ]:
def BGD(X: pd.DataFrame, y: pd.Series, num_iterations: int, theta: np.array) -> Tuple[Dict, Dict, float]:
    
    """ Perform batch gradient descent on theta parameters for the logistic regression to find minimum cost function. """

    costs = []
    for i in range(num_iterations):
        # Calculate cost and gradients 
        grads, cost = LR_CG(X, y, theta)
        # Retrieve derivatives from grads
        dtheta = grads["dtheta"]
        # Updating estimate parameters
        theta = theta - learning_rate * dtheta  
        # Record the costs
        if i % 100 == 0:
            costs.append(cost)
        # Save parameters and gradients in dictionary
        params = {"theta": theta}
        grads = {"dtheta": dtheta}
    return params, grads, costs

# Regression Predictions

In [ ]:
def predict(X: pd.DataFrame, y: pd.Series, theta_estimate: np.array, threshold: float) -> pd.Series:
    
    """ Compute regression predictions for the logistic model. """

    n = X.shape[0]
    # Initialise parameters
    y_pred = np.zeros(n)
    # Compute vector hypothesis predicting the probabilities
    h = logistic_func(X @ theta_estimate)
    # Convert probabilities y_log_prob to binary predictions
    y_pred[h>=threshold] = 1
    y_pred[h<threshold] = 0
    return y_pred


# Full Logistic Regression Model

In [ ]:
class LogisticRegression():
    
    def __init__(self, theta: np.array, num_iterations: int, learning_rate: float, threshold: float):
    
        """ Initialise parameters. """
       
        self.num_iterations = num_iterations
        self.learning_rate = learning_rate
        self.threshold = threshold
    
    def fit(self, X: pd.DataFrame, y: pd.Series) -> np.array:
    
        """ Fit logistic regression model. """

        # Batch Gradient Descent
        params, grads, costs = self.BGD(X, y)
        # Retrieve theta parameters from dictionary "parameters"
        return params["theta"]
    
    def predict(self, X: pd.DataFrame, theta_estimate: np.array) -> pd.Series:
    
        """ Compute regression predictions for the logistic model. """

        n = X.shape[0]
        # Initialise parameters
        y_pred = np.zeros(n)
        # Compute vector hypothesis predicting the probabilities
        h = self.logistic_func(X @ theta_estimate)
        # Convert probabilities y_log_prob to binary predictions
        y_pred[h>=self.threshold] = 1
        y_pred[h<self.threshold] = 0
        return y_pred
    
    def logistic_func(self, z: List) -> np.array:
        
        """ Calculate logistic/sigmoid function . """
        
        return 1/(1+np.exp(-np.array(z)))

    def LR_CG(self, X: pd.DataFrame, y: pd.Series, theta: np.array) -> Tuple[Dict, float]:
    
        """ Calculate the cost and gradients of the logistic model via gradient descent. """
        
        n = X.shape[0]
        h = self.logistic_func(X @ theta)
        # Cost function (add minus for appropriacy)
        cost = -1/n * (y.T @ np.log(h) + (1 - y).T @ np.log(1-h)) 
        # Derivatives
        dtheta = 1/n * (X.T @ (h-y))
        # Store gradients in a dictionary
        grads = {"dtheta": dtheta}
        return grads, cost

    def BGD(self, X: pd.DataFrame, y: pd.Series) -> Tuple[Dict, Dict, float]:
    
        """ Perform batch gradient descent on theta parameters for the logistic regression to find minimum cost function. """
    
        costs = []
        # Initialise theta
        theta = np.zeros(shape=(X.shape[1]), dtype=np.float32)
        for i in range(self.num_iterations):
            # Calculate cost and gradients 
            grads, cost = self.LR_CG(X, y, theta)
            # Retrieve derivatives from grads
            dtheta = grads["dtheta"]
            # Updating estimate parameters
            theta = theta - self.learning_rate*dtheta  
            # Record the costs
            if i % 100 == 0:
                costs.append(cost)
            # Save parameters and gradients in dictionary
            params = {"theta": theta}
            grads = {"dtheta": dtheta}
        return params, grads, costs

# Model Testing and Results

In [ ]:
# Create parameters
params = {"theta": np.zeros(shape=(X_train.shape[1]), dtype=np.float32), "num_iterations": 1000, "learning_rate": 0.1, "threshold": 0.5}
# Instantiate model
heart_disease_model = LogisticRegression(**params)

In [ ]:
# Fit model to training dataset to obtain estimates
theta_train = heart_disease_model.fit(X_train, y_train)
# Obtain predictions for training and test dataset
y_pred_train = heart_disease_model.predict(X_train, theta_train)
y_pred_test = heart_disease_model.predict(X_test, theta_train)
# Calculate R2 score on training and test dataset
print("Theta estimates are: \n{}".format(theta_train))
print("Train dataset accuracy: {}".format(np.float(sum(y_pred_train==y_train)) / float(len(y_train))))
print("Test dataset accuracy: {}".format(np.float(sum(y_pred_test==y_test)) / float(len(y_test))))

In [ ]:
# Print confusion matrix
cm_test = confusion_matrix(y_test, y_pred_test)
ax = sns.heatmap(cm_test, annot=True, fmt=".1f")
ax.set(xlabel="Predicted Labels", ylabel="True Labels");

In [ ]:
# Print train metric report
pd.DataFrame(classification_report(y_test, y_pred_test, output_dict=True))

**Note:** The bottom labels are the true labels and the side labels are the predicted labels.

# Summary

- It is clear that once training is complete and we use the trained model to predict the test set, logistic regression performs excellently on predicting patients that will not have a 10 year risk of heart disease (high F Score for label 0) but performs poorly and misclassifies alot of the patients who do have a 10 year risk of heart disease (low F Score for label 1 due to low recall). 
- The accuracy is very high for both training and test dataset- this is not a good representation of the model's strength however as the dataset is heavily imabalanced and thus should not be used solely as the metric of importance. 
- From the analysis above, logistic regression is perhaps therefore not the best model to use for this type of dataset (if you want to be able to correctly classify patients who do have a 10 year risk of heart disease) - either using all of the features is not a good representation to classify patients as having a 10 year heart disease or due to the high multi-dimensional feature set, we need a more 'segregative' model e.g. Random Forest or KNN. 

<hr style="height:2px;border-width:0;color:gray;background-color:gray">

## Extra

Some comments about the code implementations:

1. Here we have created an entire python class for linear regression to make it similar to a machine learning pipeline format. If you would like to just use the functions by themselves, simply take them from the individual sections.
2. For the batch gradient descent function alone, we must feed an initialised theta parameter - usually this can be just zeros. 
3. We have kept with vectorised notation as it is much faster using matrices than loops.
4. Some of the code inputs have been hidden to make the notebook neater. You may look at the code by clicking on the 'show' button. 

<hr style="height:2px;border-width:0;color:gray;background-color:gray">

Thanks for reading this notebook. If there are any mistakes or things that need more clarity, feel free to respond in the comment section and I will be happy to reply.

As always, please leave an upvote - it would also be helpful if you cite this documentation if you are going to use any of the code. 😊

#CodeWithSid